In [1]:
%cd ../../../
!python3 -m pip install --upgrade .

/home/gpadmin/GreenplumPython
Defaulting to user installation because normal site-packages is not writeable
Processing /home/gpadmin/GreenplumPython
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for greenplum-python: filename=greenplum_python-1.0.1-py3-none-any.whl size=70509 sha256=995d00c1fdf47e7721a42c1f1f1e0ffa3af7b02ea1403620ee8c2cebdacf69c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-34hu1ytc/wheels/bb/1f/99/ff8594e48ec11df99af6e0ee8611a5e560e9f44d1a3fefb351
Successfully built greenplum-python


In [2]:
content = ["I have a dog.", "I like eating apples."]

import greenplumpython as gp

db = gp.database("postgresql://localhost:7000")
t = db.create_dataframe(columns={"id": range(len(content)), "content": content})

In [3]:
import greenplumpython.experimental.embedding

gp.config.print_sql = True

t = t.save_as(
    column_names=["id", "content"], distribution_key={"id"}, distribution_type="hash"
).check_unique(columns={"id"})
t = t.embedding().create_index(column="content", model="all-MiniLM-L6-v2")
t


            CREATE  TABLE "cte_7721c0fd66c041c4b7c5ff4074b3479b"
            (id,content)
            
            AS SELECT unnest(ARRAY[0,1]) AS "id",unnest(ARRAY['I have a dog.','I like eating apples.']) AS "content"
            
                DISTRIBUTED BY (id)
            
            
WITH cte_d0f82a0d4eee4d86a93fa03651ebcb8a AS (TABLE "cte_7721c0fd66c041c4b7c5ff4074b3479b")SELECT to_json(cte_1fd18725c25a4c9a877fd6a0aa180e0a)::TEXT FROM cte_d0f82a0d4eee4d86a93fa03651ebcb8a AS cte_1fd18725c25a4c9a877fd6a0aa180e0a
----------------------------
 id | content               
----+-----------------------
  0 | I have a dog.         
  1 | I like eating apples. 
----------------------------
(2 rows)

CREATE UNIQUE INDEX ON "cte_7721c0fd66c041c4b7c5ff4074b3479b" (id)
CREATE FUNCTION "pg_temp"."func_54569d267ed6412e9026f55a5bf7601b" (content "text",model_name "text") RETURNS "vector"(384) AS $$
try:
    return GD['__func_54569d267ed6412e9026f55a5bf7601b'](content=content,model_name=mod

id,content
0,I have a dog.
1,I like eating apples.


In [4]:
t.embedding().search(column="content", query="apple", top_k=1)


            WITH embedding_oid AS (
                SELECT attrelid, attnum
                FROM pg_attribute
                WHERE 
                    attrelid = '"cte_7721c0fd66c041c4b7c5ff4074b3479b"'::regclass::oid AND
                    attname = 'content'
            ), reloptions AS (
                SELECT unnest(reloptions) AS option
                FROM pg_class, embedding_oid
                WHERE pg_class.oid = attrelid
            ), embedding_info_json AS (
                SELECT split_part(option, '=', 2)::json AS val
                FROM reloptions, embedding_oid
                WHERE option LIKE format('_pygp_emb_%s=%%', attnum)
            ), embedding_info AS (
                SELECT * 
                FROM embedding_info_json, json_to_record(val) AS (attnum int4, attrelid oid, model text)
            )
            SELECT nspname, relname, model
            FROM embedding_info, pg_class, pg_namespace
            WHERE 
                pg_class.oid = attrelid AND
 

AssertionError: Database is required to create function.